In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Carregar o dataset
base_hotel = pd.read_csv(r'/content/Hotel Reservations.csv')

# Função para classificar o preço
def classify_price(price):
    if price <= 85:
        return 1

    elif price > 85 and price < 115:
        return 2
    else:
        return 3

# Criar a coluna de label 'label_avg_price_per_room'
base_hotel['label_avg_price_per_room'] = base_hotel['avg_price_per_room'].apply(classify_price)
base_hotel = base_hotel.drop(columns=['avg_price_per_room'])

# Reorganizar as colunas
colunas = []
colunas.append('label_avg_price_per_room')
for i in range(len(base_hotel.columns[:-1])):
    print(base_hotel.columns[i])
    colunas.append(base_hotel.columns[i])

base_hotel = base_hotel[colunas]

# Lista de colunas a serem removidas
colunas_para_remover = ['Booking_ID', 'arrival_date', 'market_segment_type',
                         'repeated_guest', 'no_of_previous_cancellations',
                         'no_of_previous_bookings_not_canceled', 'booking_status']

# Filtrar e remover as colunas existentes
colunas_existentes = [col for col in colunas_para_remover if col in base_hotel.columns]
base_hotel = base_hotel.drop(columns=colunas_existentes, axis=1).copy()

# Aplicando One-Hot Encoding
base_hotel = pd.get_dummies(base_hotel, prefix=['type_of_meal_plan', 'required_car_parking_space',
                                                'room_type_reserved', 'arrival_year', 'arrival_month',
                                                'no_of_special_requests'],
                             columns=['type_of_meal_plan', 'required_car_parking_space',
                                      'room_type_reserved', 'arrival_year', 'arrival_month',
                                      'no_of_special_requests'])

# Definindo X e y
X = base_hotel.iloc[:, 1:].values
X = np.array(X).astype('float32')
y = base_hotel.iloc[:, 0].values

# Dividindo os dados em treinamento e teste
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.3, random_state=0)

X_treinamento = np.array(X_treinamento).astype('float32')
y_treinamento = np.array(y_treinamento).astype('float32')

# Definindo o modelo
modelo = RandomForestClassifier(random_state=0)

# Definindo a grade de hiperparâmetros para busca
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True]
}

# Configurando o GridSearchCV
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=3, scoring='accuracy')

# Treinando o GridSearchCV
grid_search.fit(X_treinamento, y_treinamento)

# Exibindo os melhores parâmetros encontrados
print(f'Melhores parâmetros: {grid_search.best_params_}')

# Melhor modelo após o tuning
melhor_modelo = grid_search.best_estimator_

# Fazendo previsões no conjunto de teste
y_pred = melhor_modelo.predict(X_teste)

# Avaliando o desempenho no conjunto de teste
accuracy = accuracy_score(y_teste, y_pred)
report = classification_report(y_teste, y_pred)

# Exibindo os resultados
print(f'Acurácia do melhor modelo: {accuracy * 100:.2f}%')
print('Relatório de Classificação:')
print(report)

Booking_ID
no_of_adults
no_of_children
no_of_weekend_nights
no_of_week_nights
type_of_meal_plan
required_car_parking_space
room_type_reserved
lead_time
arrival_year
arrival_month
arrival_date
market_segment_type
repeated_guest
no_of_previous_cancellations
no_of_previous_bookings_not_canceled
no_of_special_requests
booking_status
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Melhores parâmetros: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia do melhor modelo: 78.67%
Relatório de Classificação:
              precision    recall  f1-score   support

           1       0.81      0.76      0.78      3258
           2       0.74      0.77      0.75      4006
           3       0.82      0.83      0.82      3619

    accuracy                           0.79     10883
   macro avg       0.79      0.79      0.79     10883
weighted avg       0.79      0.79      0.79     10883



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb  # Adicione esta linha para importar XGBoost
from sklearn.preprocessing import LabelEncoder

# Carregar o dataset
base_hotel = pd.read_csv(r'/content/Hotel Reservations.csv')

# Função para classificar o preço
def classify_price(price):
    if price <= 85:
        return 1

    elif price > 85 and price < 115:
        return 2
    else:
        return 3

# Criar a coluna de label 'label_avg_price_per_room'
base_hotel['label_avg_price_per_room'] = base_hotel['avg_price_per_room'].apply(classify_price)
base_hotel = base_hotel.drop(columns=['avg_price_per_room'])

# Reorganizar as colunas
colunas = []
colunas.append('label_avg_price_per_room')
for i in range(len(base_hotel.columns[:-1])):
    print(base_hotel.columns[i])
    colunas.append(base_hotel.columns[i])

base_hotel = base_hotel[colunas]

# Lista de colunas a serem removidas
colunas_para_remover = ['Booking_ID', 'arrival_date', 'market_segment_type',
                         'repeated_guest', 'no_of_previous_cancellations',
                         'no_of_previous_bookings_not_canceled', 'booking_status']

# Filtrar e remover as colunas existentes
colunas_existentes = [col for col in colunas_para_remover if col in base_hotel.columns]
base_hotel = base_hotel.drop(columns=colunas_existentes, axis=1).copy()

# Aplicando One-Hot Encoding
base_hotel = pd.get_dummies(base_hotel, prefix=['type_of_meal_plan', 'required_car_parking_space',
                                                'room_type_reserved', 'arrival_year', 'arrival_month',
                                                'no_of_special_requests'],
                             columns=['type_of_meal_plan', 'required_car_parking_space',
                                      'room_type_reserved', 'arrival_year', 'arrival_month',
                                      'no_of_special_requests'])

# Definindo X e y
X = base_hotel.iloc[:, 1:].values
X = np.array(X).astype('float32')
y = base_hotel.iloc[:, 0].values

# Codificar os rótulos para começarem de 0
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir o conjunto de dados em treinamento e teste
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, y_encoded, test_size=0.3, random_state=0)

# Configuração dos hiperparâmetros para RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Número de árvores
    'max_depth': [10, 15],  # Profundidade máxima da árvore
    'learning_rate': [0.05, 0.1],  # Taxa de aprendizado
    'subsample': [0.9, 1.0],  # Proporção de amostras usadas para ajustar cada árvore
    'colsample_bytree': [0.8, 0.9],  # Proporção de colunas usadas para ajustar cada árvore
    'gamma': [0, 0.2],  # Regularização por complexidade de modelo
    'min_child_weight': [1, 3],  # Peso mínimo da folha
    'reg_alpha': [0, 0.1],  # Regularização L1
    'reg_lambda': [1, 1.5]  # Regularização L2
}

# Inicializar o classificador XGBoost
model_xgb = xgb.XGBClassifier(
    eval_metric='mlogloss',  # Métrica para problemas de classificação multiclasse
    num_class=len(np.unique(y_encoded))  # Número de classes
)

# Inicializar o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_distributions,
    n_iter=100,  # Número de combinações aleatórias a serem testadas
    scoring='accuracy',
    cv=3,  # Número de folds para cross-validation
    n_jobs=-1,  # Utiliza todos os núcleos do processador
    verbose=2,  # Mostra detalhes sobre o progresso
    random_state=0  # Para reprodutibilidade
)

# Treinar o RandomizedSearchCV
random_search.fit(X_treinamento, Y_treinamento)

# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros encontrados:\n", random_search.best_params_)

# Melhor modelo encontrado
best_model = random_search.best_estimator_

# Fazer previsões
Y_pred_xgb = best_model.predict(X_teste)

# Decodificar as previsões de volta para os rótulos originais
Y_pred_xgb_decoded = label_encoder.inverse_transform(Y_pred_xgb)

# Decodificar os rótulos de teste para os rótulos originais
Y_teste_decoded = label_encoder.inverse_transform(Y_teste)

# Avaliar o modelo
print("XGBoost Accuracy Score:", accuracy_score(Y_teste_decoded, Y_pred_xgb_decoded))
print("XGBoost Classification Report:\n", classification_report(Y_teste_decoded, Y_pred_xgb_decoded))

# Verifique as classes únicas no conjunto de teste e previsões
print("Classes únicas no conjunto de teste:", np.unique(Y_teste_decoded))
print("Classes únicas nas previsões:", np.unique(Y_pred_xgb_decoded))

Booking_ID
no_of_adults
no_of_children
no_of_weekend_nights
no_of_week_nights
type_of_meal_plan
required_car_parking_space
room_type_reserved
lead_time
arrival_year
arrival_month
arrival_date
market_segment_type
repeated_guest
no_of_previous_cancellations
no_of_previous_bookings_not_canceled
no_of_special_requests
booking_status
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Melhores hiperparâmetros encontrados:
 {'subsample': 0.9, 'reg_lambda': 1, 'reg_alpha': 0.1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 15, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9}
XGBoost Accuracy Score: 0.7957364697234218
XGBoost Classification Report:
               precision    recall  f1-score   support

           1       0.81      0.78      0.79      3258
           2       0.76      0.76      0.76      4006
           3       0.82      0.85      0.83      3619

    accuracy                           0.80     10883
   macro avg       0.80      0.80      0.80  